# ospExamples - An Example of Intel® OSPRay Techniques and Procedural Scenes
--------

Renderers require data that define the geometry, color values at the surface, lighting/illumination, and many other parameters needed for the render. This lesson will show the OSPRay two types of renderers called `scivis` and `pathtracer`.

## Learning Objectives

* Understand the specific Intel OSPRay objects set up for each renderer.
* Compare the Intel OSPRay capabilities to render a variety of scenes with different geometric and volumetric data.

***
### Renderers
Intel OSPRay has two renderers - `scivis` and `pathtracer`.

`scivis` renderer combines many rendering techniques into a single renderer. It focuses on the needs of scientific visualization and implements an OpenGL*-like material model with customizable contributions of transparency, shadows, ambient occlusion, and fully integrated volume rendering.

`pathtracer` renderer is a fully photo-realistic renderer that can be used for generating high-quality images. It is used in different fields even outside of scientific visualization.

You can choose the renderer using the `szRenderer` option in the `run.sh` cell.

***
### Scenes

Different scenes can be selected from the command-line `run.sh` script (below) and each scene corresponds to an programmatically generated objects that populate a scene. This is aimed at providing users with multiple simple scenes composed of basic geometry types, lights, volumes etc. to get started with OSPRay quickly.

Given below are different scenes listed with their string identifiers:

| Scene Number | Scene Name | Description | * |
|:---:|:---:|-----|-----|
| 0 |boxes|A simple scene with box geometry type.| <img src='../link_to_IRTK_BD/OSPRay_BD/assets_OSPRay_2/boxes.png' width=25% height=25% /> |
| 1 |cornell_box|A scene depicting a classic cornell box with quad mesh geometry type for rendering two cubes and a quad light type.| <img src='../link_to_IRTK_BD/OSPRay_BD/assets_OSPRay_2/cornell_box.png' width=25% height=25% /> |
| 2 |curves|A simple scene with curve geometry type and options to change curveBasis. For details on different basis’ please check documentation of curves.|<img src='../link_to_IRTK_BD/OSPRay_BD/assets_OSPRay_2/curves.png' width=25% height=25% /> |
| 3 |gravity_spheres_volume|A scene with structured regular type of volume.|<img src='assets/gravity_spheres_volume.png' width=25% height=25% /> |
| 4 |gravity_spheres_isosurface|A scene depicting iso-surface rendering of gravity_spheres_volume using geometry type isosurface.|<img src='../link_to_IRTK_BD/OSPRay_BD/assets_OSPRay_2/gravity_spheres_isosurface.png' width=25% height=25% /> |
| 5 |perlin_noise_volumes|An example scene with structured regular volume type depicting perlin noise.|<img src='../link_to_IRTK_BD/OSPRay_BD/assets_OSPRay_2/perlin_noise_volumes.png' width=25% height=25% /> |
| 6 |random_spheres|A simple scene depicting sphere geometry type.|<img src='../link_to_IRTK_BD/OSPRay_BD/assets_OSPRay_2/random_spheres.png' width=25% height=25% /> |
| 7 |streamlines|A scene showcasing streamlines geometry derived from curve geometry type.|<img src='../link_to_IRTK_BD/OSPRay_BD/assets_OSPRay_2/streamlines.png' width=25% height=25% /> |
| 8 |subdivision_cube|A scene with a cube of subdivision geometry type to showcase subdivision surfaces.|<img src='../link_to_IRTK_BD/OSPRay_BD/assets_OSPRay_2/subdivision_cube.png' width=25% height=25% /> |
| 9 |unstructured_volume|A simple scene with a volume of unstructured volume type.|<img src='../link_to_IRTK_BD/OSPRay_BD/assets_OSPRay_2/unstructured_volume.png' width=25% height=25% /> |
| 10 | planes | Multiple planes intersecting about the origin |<img src='assets/planes.png' width=25% height=25% /> |
| 11 | clip_with_spheres | Demonstration of using a spherical clipping |<img src='../link_to_IRTK_BD/OSPRay_BD/assets_OSPRay_2/clip_with_spheres.png' width=25% height=25% /> |
| 12 | clip_with_planes | Demonstration of using a planar volume with spheres |<img src='../link_to_IRTK_BD/OSPRay_BD/assets_OSPRay_2/clip_with_planes.png' width=25% height=25% /> |
| 13 | clip_gravity_spheres_volume | Demonstration of clipping a gravity field with spheres |<img src='../link_to_IRTK_BD/OSPRay_BD/assets_OSPRay_2/clip_gravity_spheres_volume.png' width=25% height=25% /> |
| 14 | clip_perlin_noise_volumes | Demonstration of Perlin noise visualized as a volumetric  |<img src='../link_to_IRTK_BD/OSPRay_BD/assets_OSPRay_2/clip_perlin_noise_volumes.png' width=25% height=25% /> |
    
***


## ospExamples Code
Review the source code of the ospExamples:

In [1]:
%%writefile src/ospExamples.cpp
#define _GLIBCXX_USE_CXX11_ABI 0
// rkcommon
#define OSPRAY_CPP_RKCOMMON_TYPES 

// rkcommon
#include "rkcommon/math/rkmath.h"
#include "rkcommon/math/vec.h"
#include "rkcommon/utility/ParameterizedObject.h"
#include "rkcommon/containers/TransactionalBuffer.h"
#include "rkcommon/math/AffineSpace.h"

// ospray
#include "ospray/ospray_cpp.h"
#include "ArcballCamera.h"

// ospray_testing
#include "ospray/ospray_testing/ospray_testing.h"

// GL Math library headers
#include "glm/vec2.hpp"
#include "glm/vec3.hpp"
#include "glm/vec4.hpp"
#include "glm_box3.h"

// utils
#include "utils.h"

// std
#include <iostream>
#include <stdexcept>
#include <exception>
#include <vector>
#include <functional>
#include <string>

// argh - the command line parser 
#include "argh.h"

using namespace ospray;
using namespace rkcommon::math;
using namespace std;

// Vector of strings corresponding to the different scenes listed
static const std::vector<std::string> g_scenes = {
    "boxes",
    "cornell_box",
    "curves",
    "gravity_spheres_volume",
    "gravity_spheres_isosurface",
    "perlin_noise_volumes",
    "random_spheres",
    "streamlines",
    "subdivision_cube",
    "unstructured_volume",
    "planes",
    "clip_with_spheres",
    "clip_with_planes",
    "clip_gravity_spheres_volume",
    "clip_perlin_noise_volumes"
};
                
// ######################################################################
// Entry point to the program

int main(int argc, const char* argv[]) try {

    // setting up the command line parser: Inputs from the command line when the program ospExamples.cpp is called
    auto parser = argh::parser(argc, argv);
    parser.parse(argc, argv, argh::parser::PREFER_PARAM_FOR_UNREG_OPTION);
    
    // setting the szRenderer variable parsed from the command-line
    std::string szRenderer;
    parser({"-szRenderer","--szRenderer"}, "scivis") >> szRenderer;
    
    // setting the nScene variable parsed from the command-line
    int nScene;
    parser({ "-nScene", "--nScene"}, 0) >> nScene;
    
    // Full initialization of the OSPRay renderer/device
    utils::initializeOSPRay(argc, argv);
    
    // image size
    vec2i imgSize;
    imgSize.x = 1024; // width
    imgSize.y =  768; // height

    // OSPRay objects 
    cpp::Renderer renderer;
    cpp::Camera   camera{"perspective"};
    cpp::World    world;

    // Arcball camera instance
    std::unique_ptr<ArcballCamera> arcballCamera;

    vec4f backgroundColor { 0.25f, 0.25f, 0.25f, 1.f };

    // a running list of OSPRay objects to commit
    rkcommon::containers::TransactionalBuffer<OSPObject> objectsToCommit;

    auto builder = testing::newBuilder(g_scenes[nScene]);
    testing::setParam(builder, "rendererType", szRenderer);
    testing::commit(builder);

    world = testing::buildWorld(builder);
    testing::release(builder);

    world.commit();

    // creating the renderer object - either scivis or pathtracer
    renderer = cpp::Renderer(szRenderer.c_str());
    
    // retains a set background color on renderer change
    renderer.setParam("backgroundColor", backgroundColor);
    if(szRenderer == "sciviz") {
        renderer.setParam("aoSamples", 16);
        renderer.setParam("aoRadius", 100.f);
        renderer.setParam("aoIntensity", 1.f);
        renderer.setParam("volumeSamplingRate", 1.f);
    } else if(szRenderer == "pathtracer") {
        renderer.setParam("lightSamples",1);
        renderer.setParam("geometryLights",true);
        renderer.setParam("roulettePathLength",5);
        renderer.setParam("maxContribution", 10.f);
    }
    renderer.commit();

    // create the arcball camera model
    arcballCamera.reset(new ArcballCamera(world.getBounds<box3f>(), imgSize));
    arcballCamera->updateWindowSize(imgSize);
    arcballCamera->rotate(vec2f(0.f), vec2f(0.25f));
    
    camera.setParam("aspect",    imgSize.x / float(imgSize.y));
    camera.setParam("position",  arcballCamera->eyePos());
    camera.setParam("direction", arcballCamera->lookDir());
    camera.setParam("up",        arcballCamera->upDir());
    camera.commit();

    // create and setup framebuffer. The framebuffer holds the rendered 2D image.
    // with arguments as x and y size, format, color and channel of the picture
    // These are the picture formats allowed:
        // OSP_FB_NONE:    framebuffer will not be mapped by the application
        // OSP_FB_RGBA8:   bit [0–255] linear component red, green, blue, alpha
        // OSP_FB_SRGBA:   bit sRGB gamma encoded color components, and linear alpha
        // OSP_FB_RGBA32F: 32 bit float components red, green, blue, alpha
    ospray::cpp::FrameBuffer framebuffer(
        imgSize.x, imgSize.y, OSP_FB_SRGBA, OSP_FB_COLOR | OSP_FB_ACCUM);
    framebuffer.clear();

    // render one frame
    framebuffer.renderFrame(renderer, camera, world);

    // access framebuffer and write its content as PNG file
    uint32_t *fb = (uint32_t *)framebuffer.map(false ? OSP_FB_ALBEDO : OSP_FB_COLOR);

    utils::writePNG("boxes.png", glm::ivec2(1024, 768), fb);
    
    framebuffer.unmap(fb);

    return 0;
} catch( std::exception &ex ) {
  std::cout << ex.what() << std::endl;
}

Overwriting src/ospExamples.cpp


***
## Build the program

In [3]:
! ./build.sh

## ue67fcfb20b4827a1ed6842c10ec58c1 is compiling ospExamples.cpp

CommandNotFoundError: Your shell has not been properly configured to use 'conda deactivate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.



CommandNotFoundError: Your shell has not been properly configured to use 'conda deactivate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C com

***
## Execute the program
You can change the type of renderer used by setting the `szRenderer` command line option to `pathtracer` or `scivis`.

You can also select a different scene to render by setting the `nScene` command line option to a number from 0 to 14. The number corresponds to a scene index entry from the table listing above.

For example, to run the program with the `pathfinder` renderer for a boxes scene:

In [5]:
! ./run.sh --szRenderer=pathtracer --nScene=2

## ue67fcfb20b4827a1ed6842c10ec58c1 is running ospExamples
./run.sh: line 8: bin/ospExamples: No such file or directory


***
## View the Resulting Image

***
## Summary
You have arrived at the end of this lesson. During this lesson, you:

* Inspected the code and reviewed the comments to understand the specific Intel OSPRay objects set up for each renderer.
* Compared the Intel OSPRay capabilities to render a variety of scenes with different geometric and volumetric data.


***
<html><body><span style="color:green"><h1>Next: Intel&reg; OSPRay Asset Loading - Loading an OBJ/MTL File</h1></span></body></html>

[Click Here](../osp3_asset_load/asset_loader.ipynb)

<html><body><span style="color:green"><h1>Back: Overview</h1></span></body></html>

[Click Here](../Overview.ipynb)